Importación y Carga de Datos

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE

# Cargar el dataset
df = pd.read_csv('Telco Churn dataset 2.csv')

Preprocesamiento

In [10]:
# 1. Preprocesamiento
# a. Crear la columna 'Churn' y asignar 1 si 'Churn' es 'Yes', de lo contrario 0
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# b. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# c. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    if col != 'Churn':  # Asegurarse de no modificar la columna 'Churn'
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# d. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Churn':
        df[column].fillna(df[column].mode()[0], inplace=True)
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)

# e. Normalización
cols_to_scale = df.columns.tolist()
cols_to_scale.remove('Churn')
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# f. Asegurarse de que 'Churn' sea int
df['Churn'] = df['Churn'].astype(int)


In [11]:
churn_counts = df['Churn'].value_counts()
print("Cantidad de registros antes del preprocesamiento:")
print("Churn = 0:", churn_counts[0])
print("Churn = 1:", churn_counts[1])

Cantidad de registros antes del preprocesamiento:
Churn = 0: 2850
Churn = 1: 483


Selección de características

In [12]:
# 2. Selección de características
xgb_for_feature_selection = XGBClassifier(
    objective='binary:logistic', 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_for_feature_selection.fit(df.drop('Churn', axis=1), df['Churn'])
threshold = 0.01  
selected_features = df.drop('Churn', axis=1).columns[(xgb_for_feature_selection.feature_importances_ > threshold)].tolist()
print("Características seleccionadas:", selected_features)

Características seleccionadas: ['customerID', 'SeniorCitizen', 'tenure', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'InternationalPlan', 'VoiceMailPlan', 'NumbervMailMessages', 'TotalDayMinutes', 'TotalEveMinutes', 'TotalNightMinutes', 'TotalIntlMinutes', 'TotalIntlCalls', 'CustomerServiceCalls']


In [13]:
print("Número de características seleccionadas:", len(selected_features))

Número de características seleccionadas: 22


In [14]:
print("Antes del balanceo:")
churn_counts = df['Churn'].value_counts()
print("Cantidad de registros antes del Balanceo:")
print("Churn = 0:", churn_counts[0])
print("Churn = 1:", churn_counts[1])

Antes del balanceo:
Cantidad de registros antes del Balanceo:
Churn = 0: 2850
Churn = 1: 483


Balanceo con SMOTE

In [15]:
# 3. Balanceo con SMOTE
X = df[selected_features]
y = df['Churn']
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [16]:
# Imprimir el número de registros después del balanceo
print("Después del balanceo:")
print("Número total de registros:", len(y_smote))
print("Número de registros con etiqueta 1:", sum(y_smote == 1))
print("Número de registros con etiqueta 0:", sum(y_smote == 0))

Después del balanceo:
Número total de registros: 5700
Número de registros con etiqueta 1: 2850
Número de registros con etiqueta 0: 2850


División de Conjunto

In [17]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [19]:
print("Después de División:")
print("Número total de registros:", len(y_train))
print("Número de registros con etiqueta 1 entrenamiento:", sum(y_train == 1))
print("Número de registros con etiqueta 0 entrenamiento:", sum(y_train == 0))
print("Número total de registros:", len(y_test))
print("Número de registros con etiqueta 1 Test:", sum(y_test == 1))
print("Número de registros con etiqueta 0 Test:", sum(y_test == 0))

Después de División:
Número total de registros: 4560
Número de registros con etiqueta 1 entrenamiento: 2285
Número de registros con etiqueta 0 entrenamiento: 2275
Número total de registros: 1140
Número de registros con etiqueta 1 Test: 565
Número de registros con etiqueta 0 Test: 575


Selección de Hiperparámetros

In [12]:
# 5. Entrenamiento de XGBoost con optimización de hiperparámetros
param_dist = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.3],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.25, 0.5, 1.0],
    'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
    'scale_pos_weight': [1, 3, 5]
}

xgb_model = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=150, cv=5, verbose=1, n_jobs=-1, random_state=42)
xgb_search.fit(X_train_smote, y_train_smote, early_stopping_rounds=10, eval_set=[(X_test_selected, y_test)], verbose=False)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:885:

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           random_state=42, ...),
                   n_iter=150, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1.0],
                                        'gamma': [0, 0.25, 0.5, 1.0],
                                        'learning_rate': [0.01, 0.03, 0.05, 0.1,
                                                          0.3],
                                        'max_depth': [3, 4, 5, 6, 7, 8],
                                        'n_estimators': [100, 300, 500, 800,
                                                         1000],
                                        'reg_lambda': [0.1, 1.0, 5.0, 10.0,
                                                       50.0, 100.0],
                                        'scale_pos_weight': [1, 3, 5],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42, verbose=1)

Evaluación del modelo

In [13]:
# 6. Evaluación del modelo
y_pred = xgb_search.best_estimator_.predict(X_test_selected)
conf_matrix = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(conf_matrix)
print("Accuracy con XGBoost:", acc)
print(report)

[[849  86]
 [ 78 305]]
Accuracy con XGBoost: 0.8755690440060698
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       935
           1       0.78      0.80      0.79       383

    accuracy                           0.88      1318
   macro avg       0.85      0.85      0.85      1318
weighted avg       0.88      0.88      0.88      1318



Validación Cruzada

In [14]:
# 7. Validación Cruzada
scores = cross_val_score(xgb_search.best_estimator_, X_train_smote, y_train_smote, cv=5, scoring='accuracy')
print("Accuracy promedio con Validación Cruzada:", scores.mean())

Accuracy promedio con Validación Cruzada: 0.903434610303831


In [16]:
print(xgb_search.best_params_)

{'subsample': 1.0, 'scale_pos_weight': 1, 'reg_lambda': 1.0, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.9}
